In [13]:
### Import necessary libraries
import time
import random
import tqdm
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException, WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.common.by import By
from selenium.webdriver import Firefox
from urllib.parse import urlparse, parse_qsl, urlencode, urlunparse
import json
import pickle


# Funciones para el proyecto

In [45]:
def get_random_user_agent():
    HEADERS_LIST = [
        {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,/;q=0.8",
            "Accept-Language": "en-US,en;q=0.9",
            "Referer": "https://www.google.com/",
            "Connection": "keep-alive",
            "Cookie": "CONSENT=YES+1; NID=511=exampleSessionId123abc"
        },
        {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,/;q=0.8",
            "Accept-Language": "en-US,en;q=0.8",
            "Referer": "https://duckduckgo.com/",
            "Connection": "keep-alive",
            "Cookie": "D=do_not_track; example_cookie_id=macSafariSession987"
        },
        {
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9",
            "Accept-Language": "en-GB,en;q=0.8",
            "Referer": "https://www.bing.com/",
            "Connection": "keep-alive",
            "Cookie": "session=linux_session_xyz789; logged_in=true"
        },
        {
            "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 17_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,/;q=0.8",
            "Accept-Language": "en-US,en;q=0.9",
            "Referer": "https://m.facebook.com/",
            "Connection": "keep-alive",
            "Cookie": "fb_locale=en_US; datr=mobileSafariFakeSession123"
        }
    ]
    return random.choice(HEADERS_LIST)

In [57]:
### Set del driver y lista de user-agents
USER_AGENTS = [
        {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.6312.86 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,/;q=0.8",
            "Accept-Language": "en-US,en;q=0.9",
            "Referer": "https://www.google.com/",
            "Connection": "keep-alive",
            "Cookie": "CONSENT=YES+1; NID=511=exampleSessionId123abc"
        },
        {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.3 Safari/605.1.15",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,/;q=0.8",
            "Accept-Language": "en-US,en;q=0.8",
            "Referer": "https://duckduckgo.com/",
            "Connection": "keep-alive",
            "Cookie": "D=do_not_track; example_cookie_id=macSafariSession987"
        },
        {
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9",
            "Accept-Language": "en-GB,en;q=0.8",
            "Referer": "https://www.bing.com/",
            "Connection": "keep-alive",
            "Cookie": "session=linux_session_xyz789; logged_in=true"
        },
        {
            "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 17_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,/;q=0.8",
            "Accept-Language": "en-US,en;q=0.9",
            "Referer": "https://m.facebook.com/",
            "Connection": "keep-alive",
            "Cookie": "fb_locale=en_US; datr=mobileSafariFakeSession123"
        }
    ]

def setup_driver(headless=False):
    """Inicializa el WebDriver de Selenium con configuración de Firefox en modo headless."""
    profile = FirefoxProfile()
    profile.set_preference("general.useragent.override", get_random_user_agent())
    profile.set_preference("intl.accept_languages", "es-ES,es")
    firefox_options = Options()
    firefox_options.profile = profile
    if headless:
        firefox_options.add_argument("--headless")  # Run in headleßss mode
    driver = Firefox(options=firefox_options)
    driver.set_page_load_timeout(10)
    return driver


In [81]:
def setup_driver(headless=False):
    headers = get_random_user_agent()

    profile = FirefoxProfile()
    profile.set_preference("general.useragent.override", headers["User-Agent"])
    profile.set_preference("intl.accept_languages", headers["Accept-Language"])

    options = Options()
    options.profile = profile
    if headless:
        options.add_argument("--headless")

    driver = Firefox(options=options)
    driver.set_page_load_timeout(10)
    
    # Navegar desde un referer simulado
    driver.execute_script(f"window.location.href = '{headers['Referer']}';")

    return driver

In [49]:
### Sistema anti-bot (comportamiento humano)
def simulate_human_behavior(min_delay=1, max_delay=3, verbose=False):
    """Simula comportamiento humano con sleeps aleatorios."""
    delay = round(random.uniform(min_delay, max_delay), 2)
    if verbose:
        print(f"Esperando {delay}s...")
    time.sleep(delay)


In [51]:
### Obtener lista de practice areas para aplicar filtros
def extract_trimmed_set(series):
    """
    Recibe una Serie de pandas con valores separados por comas,
    elimina NaN, divide por ',' y devuelve un set de elementos únicos y limpios.
    """
    values = series.dropna().astype(str)
    trimmed_set = set()
    for val in values:
        items = [item.strip() for item in val.split(',')]
        trimmed_set.update(items)
    return trimmed_set


In [52]:
### Obtener paginas totales por cada busqueda una vez aplicado el filtro 
def generate_paginated_urls(url,driver,verbose=False):
    # Encontrar numero de paginas 
    try:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR,'ul.right')))
        simulate_human_behavior(1,2,verbose=False)
        pages_tab = driver.find_element(By.CSS_SELECTOR,'ul.right')       
        if pages_tab:
            pages = pages_tab.find_elements(By.TAG_NAME,"li")
            pages = pages[1].text
            pages = int(pages.split()[-1])
    except Exception as e: 
        #print(f"Solo hay una pagina")
        pages = 1

    #print(f"total pages:{pages}")
    if verbose:
        print(f"paginas en esta url: {pages}")
    parsed = urlparse(url)
    query_list = parse_qsl(parsed.query)  # esto mantiene el orden de los parámetros

    urls = []
    for i in range(1, pages + 1):
        # Creamos una nueva lista sin el parámetro page
        new_query = [(k, v) for (k, v) in query_list if k != "page"]
        # Si no es la primera página, agregamos page=N
        if i != 1:
            new_query.append(("page", str(i)))
        # Reconstruimos la query string manteniendo el orden
        new_query_string = urlencode(new_query)
        # Reconstruimos la URL
        new_url = urlunparse((parsed.scheme, parsed.netloc, parsed.path, '', new_query_string, ''))
        urls.append(new_url)

    return urls


In [53]:
### crawler
def crawler(url, driver,verbose=False,get_driver=True):   
    try:
        list_urls = []
        urls_unicos = set()  # Usar un set para almacenar URLs únicas
        if get_driver:
            driver.get(url)
        time.sleep(1)
        # buscar contenedor con info
        WebDriverWait(driver, 25).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.results__title')))
        result_title = driver.find_element(By.CSS_SELECTOR, '.results__title')
        if result_title:
            WebDriverWait(driver, 25).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.results__result-row'))) 
            attorney_table = driver.find_element(By.CSS_SELECTOR, '.results__result-row')
            abogados = attorney_table.find_elements(By.XPATH, '(//div[@class="row collapse card__content"])')
            if verbose:
                print("se encontro la tabla de abogados")
                print(len(abogados))
                print(abogados[0])

            for ul in abogados:
                try:
                    WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.TAG_NAME, 'ul')))
                    enlaces = ul.find_elements(By.TAG_NAME, 'a')
                    href_attorney = enlaces[1].get_attribute('href')         # Obtiene el atributo href
                    if verbose:
                            print(href_attorney)
                    if href_attorney not in urls_unicos:
                        urls_unicos.add(href_attorney)
                        list_urls.append(href_attorney)
                except:
                    print("No se encontró <a> en este <li>")
                    pass
        if verbose:
            print(driver.current_url)
            print(len(list_urls))
    except:
        print("problemas con el crawler")
    return list_urls 


In [54]:
### función para obtener el HTML
def get_html(driver,url,verbose=False):
    driver.get(url)
    time.sleep(random.uniform(3, 5))
    if verbose:
        print(f"accediendo a: {url}")
    WebDriverWait(driver, 20).until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, 'body > div.off-canvas-wrap > div > div.attorney-profile-content.profile-content')))
    simulate_human_behavior(1, 2, verbose=False)
    html = driver.page_source
    return html


In [114]:
### funcion Scraper pagina por pagina abogados
def scrape_abogados(lista_urls):
    resultados = []
    count = 902 ### CAMBIAR LUEGO
    for url in lista_urls:
        driver = setup_driver(headless=True)
        info_attorney = {}
        try:
            print(f"accediendo a {count}: {url}")
            html = get_html(driver,url)
            soup = BeautifulSoup(html, 'html.parser')

            profile_content = soup.select_one('div.attorney-profile-content.profile-content')
            if not profile_content:
                print(f"Perfil no encontrado para {url}")
                pass

            # Obtencion de elementos princiaples de cada pagina
            attorney_card = profile_content.select_one(".masthead-content")
            firm = attorney_card.find("span")
            if firm:
                firm = firm.get_text(strip=True)
            url_martindale_firm = attorney_card.find("a")["href"]

            # En caso de que no haya un link de firma, el primer link sera la caja de rewiew que redirige a la misma pagina, por eso el #
            if "#" in url_martindale_firm:
                url_martindale_firm = None

            # Estructura del diccionario
            info_attorney["url_martindale"] = url #URL interna de martindale para cada abogado
            info_attorney["name"] = profile_content.find('h1').get_text(strip=True) #Nombre del abogado
            info_attorney["firm"] = firm
            info_attorney["url_firm_martindale"] = url_martindale_firm

            # Busqueda de la sección de links de contacto, no todas las paginas tienen esta sección. por eso se usa try/except 
            try: 
                web_bar = profile_content.select_one("section.sticky-menu-all:nth-child(2) > nav:nth-child(1)")
                website_firm = web_bar.select('a')[1]["href"]
                info_attorney["website_firm"] = website_firm if website_firm else None
            except Exception as e:
                #print(f"Error al extraer website: {e}")
                info_attorney["website"] = ""

            #Extraccion de los tags relevantes de cada pagina
            main_content_table = soup.find_all("div", class_="small-12 columns")[0]
            for section_div in main_content_table.find_all("div", recursive=False):
                content_div = section_div.find("div", class_="toggle-area__content")
                h2_key = section_div.find("h2") #Titulo de sección por separado
                if h2_key and content_div:
                    h2_key = h2_key.get_text(strip=True)
                    if "Areas of Practice" in h2_key:
                        h2_key = "Areas of Practice"
                    #print(h2_key)
                    info_attorney[h2_key] = "" #inicializar keys que se encuentran

                    # Biography and practice areas 
                    child_text = content_div.find('div', class_='truncate-text')
                    if child_text:
                        info_attorney[h2_key] = child_text.get_text(strip=True)
                    Aoe_area = content_div.find('div', class_='row collapse')
                    if Aoe_area:
                        ul_info = Aoe_area.find('ul')
                        if ul_info:
                            items = [li.get_text(strip=True) for li in ul_info.find_all('li')]
                            info_attorney[h2_key] = ', '.join(items)

                    # Education and credentials
                    EnC_area = content_div.find_all(
                        lambda tag: tag.name == 'div' and 
                        all(cls in tag.get('class', []) for cls in ['row', 'collapse', 'experience-section', 'clearfix'])
                    )
                    for text in EnC_area:
                        text_info = text.get_text(strip=True)
                        for line in text_info.split('\n'):
                            if ':' in line:
                                k, v = map(str.strip, line.split(":", 1))
                                info_attorney[k] = v
                            else:
                                info_attorney[h2_key] = line.strip()

                    # Peer rewiew - Pendiente
                    PrA_section = content_div.find("section", class_="profile-peer-reviews")
                    if PrA_section: 
                        rewiew_area = PrA_section.find('div', class_="review-top review-top-peer-reviews")
                        rewiew_list = PrA_section.find('div', class_="attorney-reviews-list attorney-peer-reviews-list")
                        if rewiew_area and rewiew_list:
                            text_rewiew_area = rewiew_area.find_all('span')[2].get_text().strip().splitlines()
                            stars = text_rewiew_area[0].split("/")[0].strip()   
                            reviews = text_rewiew_area[1].strip("()").split()[0]
                            info_attorney["profile_peer_review_star"] = stars
                            info_attorney["profile_peer_review_count"] = reviews

                    # Location
                    firm_offices = content_div.find('ul', id='firmOffices')
                    if firm_offices:
                        location_items = firm_offices.find_all('div', class_='office-address')
                        if location_items:
                            info_attorney["Location"] = location_items[0].get_text(strip=True)
            if info_attorney:
                resultados.append(info_attorney)
            count += 1 
            driver.quit()
        except TimeoutException:
            print("Loading took too much time!")
            print(f"error en iter {count}")
            driver.quit()
            return resultados
        except Exception as e:
            print(f"Failed to load page due to error: {e}")
            print(f"error en iter {count}")
            driver.quit()
            return resultados
        except NoSuchElementException:
            driver.quit()
            print(f"error en iter {count}")
            return resultados
        driver.quit()
    # Resultado ya es una lista de diccionarios
    return resultados


## Primera iteración - Obtención lista "practice areas"

In [159]:
#all_info = pd.DataFrame() # Data frame para almacenar información
url_inicial = 'https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto%20Rico%2C%20USA'
lista_urls = []


In [ ]:
urls_per_filter = []  # se inicializa los links que hay por filtro o busqueda. 
datos_preliminares = []


In [261]:
### Lista de Areas of Practice para emplear en filtros 
try:
    # Setup del driver
    driver = setup_driver()
    driver.get(url_inicial)
    pages = generate_paginated_urls(url_inicial,driver)
    # Crawler inicial para obtener los urls de abogados de la primera pagina
    for url in pages: #se obtiene el numero de paginas en cada busqueda
        simulate_human_behavior(1,2,verbose=False)
        links_per_page = crawler(url,driver,verbose=False) #deberian ser 25 links de abogados por cada pagina de busqueda
        if len(links_per_page) == 0:
            urls_per_page = crawler(url,driver,verbose=False)
            time.sleep(2)
        else:
            urls_per_filter = urls_per_filter + links_per_page
except TimeoutException:
    print("Loading took too much time!")
    driver.quit()
except Exception as e:
    print(f"Failed to load page due to error: {e}")
    driver.quit()
except NoSuchElementException:
    driver.quit()
driver.quit()


Failed to load page due to error: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:199:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:747:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:559:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:1071:39
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20



In [ ]:
urls_per_filter
#len(urls_per_filter)
with open("urls.pkl", "wb") as f:
    pickle.dump(urls_per_filter, f) #Primera pagina "sin filtros de practice area aplicados"


In [253]:
with open("urls.pkl", "rb") as f:
    urls_per_filter = pickle.load(f) # Carga de urls de primera iteración, ya la lista no es 0. 
urls_per_filter # Toca cambiar el nombre a url_complete o algo así porque va almacenar todo


['https://www.martindale.com/attorney/tessie-leal-garabis-esq-169104324/',
 'https://www.martindale.com/attorney/ramon-mellado-gonzalez-1572913/',
 'https://www.martindale.com/attorney/edgardo-j-hernandez-oharriz-esq-158870123/',
 'https://www.martindale.com/attorney/jaime-j-aponte-parsi-1573642/',
 'https://www.martindale.com/attorney/vicente-a-sequeda-torres-1572418/',
 'https://www.martindale.com/attorney/ross-alan-miller-300844400/',
 'https://www.martindale.com/attorney/david-j-fernandez-esteves-1571798/',
 'https://www.martindale.com/attorney/carlos-villegas-del-valle-301593670/',
 'https://www.martindale.com/attorney/hector-pedrosa-41531869/',
 'https://www.martindale.com/attorney/karla-m-rodriguez-morales-157447904/',
 'https://www.martindale.com/attorney/ms-helaine-a-gregory-321442/',
 'https://www.martindale.com/attorney/joseph-d-lento-168969230/',
 'https://www.martindale.com/attorney/glenn-carl-james-1573988/',
 'https://www.martindale.com/attorney/benjamin-acosta-jr-157179

In [ ]:
### obtencion de lista de practice area 
data_for_PA = scrape_abogados(urls_per_filter) # Primera iteración. se obtuvieron 500 registros iniciales


accediendo a 1: https://www.martindale.com/attorney/tessie-leal-garabis-esq-169104324/
accediendo a 2: https://www.martindale.com/attorney/ramon-mellado-gonzalez-1572913/
accediendo a 3: https://www.martindale.com/attorney/edgardo-j-hernandez-oharriz-esq-158870123/
accediendo a 4: https://www.martindale.com/attorney/jaime-j-aponte-parsi-1573642/
accediendo a 5: https://www.martindale.com/attorney/vicente-a-sequeda-torres-1572418/
accediendo a 6: https://www.martindale.com/attorney/ross-alan-miller-300844400/
accediendo a 7: https://www.martindale.com/attorney/david-j-fernandez-esteves-1571798/
accediendo a 8: https://www.martindale.com/attorney/carlos-villegas-del-valle-301593670/
accediendo a 9: https://www.martindale.com/attorney/hector-pedrosa-41531869/
accediendo a 10: https://www.martindale.com/attorney/karla-m-rodriguez-morales-157447904/
accediendo a 11: https://www.martindale.com/attorney/ms-helaine-a-gregory-321442/
accediendo a 12: https://www.martindale.com/attorney/joseph-d

In [ ]:
len(data_for_PA)

#len(urls_per_filter)
with open("data_attornies.pkl", "wb") as f:
    pickle.dump(data_for_PA, f) # Almacenamiento de la lista de datos de la primera corrida (ya son los primeros 500)


In [242]:
with open("data_attornies.pkl", "rb") as f:
    data_for_PA = pickle.load(f) # Carga de lista de diccionarios de abogados de primera iteración, ya la lista no es 0.
    # Esta variable va a ser mas general por lo que se va a emplear mas adelante. o asignar a la primera corrida. 
len(data_for_PA)

499

In [ ]:
# Creación de set() de practice tomadas de la primera iteración de abogados - se toma de la lista de diccionarios 
# se toman los practice areas y se obtienen las coinciedencias unicas antes de hacer una validación en el site
practice_areas = set()
# Recorrer cada diccionario
for entry in data_for_PA:
    areas = entry.get("Areas of Practice", "")
    # Separar por coma, quitar espacios y agregar al set
    for area in areas.split(","):
        area = area.strip()
        if area:
            practice_areas.add(area)
with open("practice_area_set_preclean.pkl", "wb") as f:
    pickle.dump(practice_areas, f)
print(len(practice_areas))


589


In [ ]:
### Limpieza set Areas of practice - insumo para nuevo crowler
# Esta parte puede ser una función que se aplique antes de entrar a hacer las busquedas 

clean = [] # Lista con las practice areas que si disparan una coincidencia en el buscador. 
len(practice_areas) # Toma el set de todas las posibles practice areas, escritas por abogado
url_base = 'https://www.martindale.com/search/attorneys/?term=Puerto%20Rico%2C%20USA' ## CAMBIAR MAS ADELANTE
driver = setup_driver()
wait = WebDriverWait(driver, 15)
driver.get(url_base)

for area in practice_areas: 
    
    simulate_human_behavior(0.5,2,verbose=False)
    ### Elementos clickeables y request
    wait.until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="filtersContainer"]'))) 
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="practiceAreas"]')))
    #print("es clickeable")
    searchBar = driver.find_element(By.ID, 'practiceAreas')
    searchBar.clear()
    searchBar.send_keys(area)

    simulate_human_behavior(1,2,verbose=False)
    suggestion = wait.until(EC.visibility_of_element_located(
            (By.CSS_SELECTOR, "ul.ui-autocomplete li.ui-menu-item div.ui-menu-item-wrapper")
        ))
        ## Busqueda de practice area
    #print("Sugerencia encontrada:", suggestion.text)
    if "No Areas Of Practice matching" in suggestion.text:   
            pass
    else:
        clean.append(area)
        print(area)
    ## Numero de paginas
driver.quit()
len(clean)


Consular Law
Public Finance
Wills
Project Finance
Drug Crimes
Construction Accidents
Estate Litigation
Finance
Divorce
Boating Accidents
Government Contracts
Complex Litigation
Felonies
Citizenship
Construction Law
rico
Workers Compensation
Criminal Law
Slip and Fall
Administrative
Civil Practice
Personal Injury
Family
Federal Criminal Law
Restraining Order
Foreclosures
Intellectual Property
Mediation
Residential Real Estate
Franchises and Franchising
Corporate
Nonprofit and Charitable Organizations
Business Litigation
Education Law
Construction
Sex Crimes
Tort
Commercial
Negligence
Trusts and Estates
Natural Resources
Consumer Law
Telecommunications Law
Appellate Practice
Bankruptcy Chapter 11
Trademarks
ERISA
Corporate Tax
Municipal Law
Copyrights
Family law
Technology and Science
Collection
Commercial litigation
Banking
Collections
Eminent Domain
Torts
Business Law
International Trade
Asset Protection
Environmental Law
Sports Law
Banking Law
Labor and Employment
Inheritance
Medical 

157

In [244]:
# Almacenamiento Practice Areas unicas para filtrado. 
with open("practice_area_set.pkl", "wb") as f:
    pickle.dump(clean, f)

In [280]:
### Extracción multiples urls abogados aplicando filtros con set de "practice area" que si funcionan generan sugerencias en el buscador

# clean va a ser una lista de practice area que ya paso por un proceso de filtrado. 
#url_base = 'https://www.martindale.com/search/attorneys/?term=Puerto%20Rico%2C%20USA' #Va a cambiar mas adelante
url_base = 'https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto%20Rico%2C%20USA&loc=1'
try:
    for area in tqdm.tqdm(clean[66:]):
        ### configuración del webdriver
        driver = setup_driver(headless=True)
        wait = WebDriverWait(driver, 25)
        driver.get(url_base)
        simulate_human_behavior(0.5,3,verbose=False)

        ### Elementos clickeables y request
        # Ubicar barra de busqueda "Filters - Practice Areas"
        wait.until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="filtersContainer"]'))) 
        wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="practiceAreas"]')))
        searchBar = driver.find_element(By.ID, 'practiceAreas')
        searchBar.clear()
        
        # Envio de texto "practice area" para aplicar filtro y tener nuevos resultados
        searchBar.send_keys(area)
        simulate_human_behavior(1,2,verbose=False)
        suggestion = wait.until(EC.visibility_of_element_located(
            (By.CSS_SELECTOR, "ul.ui-autocomplete li.ui-menu-item div.ui-menu-item-wrapper")))

        # Busqueda de practice area - Si hay una sugerencia, se da click. 
        suggestion.click() # ----> Entra a nueva pagina con filtro.
        simulate_human_behavior(1,2,verbose=False) 
        current_url = driver.current_url # Actualizar la url para obtener N° de paginas y urls de paginas
        
        # links y N° de paginas una vez aplicado el filtro practice area
        urls = generate_paginated_urls(current_url, driver,verbose=True)
        for url in urls:
            print(f"accediendo a: {url}")
            urls_per_page = crawler(url,driver,verbose=False)
            if len(urls_per_page) == 0:
                urls_per_page = crawler(url,driver,verbose=False)
                time.sleep(2)
            if len(urls_per_page) > 1:
                for link in urls_per_page:
                    if not link in urls_per_filter:
                        urls_per_filter.append(link)
                        print(link)
                    else: 
                        pass
                # La lista urls_per_filter ya tiene 500 valores debido a la primera iteración del proceso. 
                # despues de cada filtro, hay links repetidos, se intenta que no hayan duplicados y sea la lista total por estado. 
        driver.quit()
except TimeoutException:
    print("Loading took too much time!")
    driver.quit()
except Exception as e:
    print(f"Failed to load page due to error: {e}")
    driver.quit()
driver.quit()
print(len(urls_per_filter))


  0%|          | 0/91 [00:00<?, ?it/s]

paginas en esta url: 5
accediendo a: https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto+Rico%2C+USA&params=eyJ0eXBlIjoicGVvcGxlIiwiY29tcGFyZSI6IiIsInRlcm0iOiJQdWVydG8gUmljbywgVVNBIiwiZ2VvTG9jYXRpb25GYWNldCI6WyJQdWVydG8gUmljbywgVS5TLkEuIl0sImdlb0xvY2F0aW9uRmFjZXRSZWNlbnRzIjpbIlB1ZXJ0byBSaWNvLCBVLlMuQS4iXSwicGFnZSI6MSwibGltaXQiOjI1LCJvZmZzZXQiOjAsInNvcnQiOiIiLCJzb3J0VHlwZSI6IiIsImNsZWFyUGFyYW1zIjpmYWxzZSwia2V5d29yZCI6IiIsInBhZ2VUaXRsZSI6Imw6UHVlcnRvIFJpY28sIFUuUy5BLiIsInByYWN0aWNlQXJlYXMiOlsiTWVkaWNhbCBNYWxwcmFjdGljZSJdfQ%3D%3D
accediendo a: https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto+Rico%2C+USA&params=eyJ0eXBlIjoicGVvcGxlIiwiY29tcGFyZSI6IiIsInRlcm0iOiJQdWVydG8gUmljbywgVVNBIiwiZ2VvTG9jYXRpb25GYWNldCI6WyJQdWVydG8gUmljbywgVS5TLkEuIl0sImdlb0xvY2F0aW9uRmFjZXRSZWNlbnRzIjpbIlB1ZXJ0byBSaWNvLCBVLlMuQS4iXSwicGFnZSI6MSwibGltaXQiOjI1LCJvZmZzZXQiOjAsInNvcnQiOiIiLCJzb3J0VHlwZSI6IiIsImNsZWFyUGFyYW1zIjpmYWxzZSwia2V5d29yZCI6IiIsInBhZ2VUaXRsZSI6Im

  1%|          | 1/91 [00:16<24:46, 16.52s/it]

paginas en esta url: 2
accediendo a: https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto+Rico%2C+USA&params=eyJ0eXBlIjoicGVvcGxlIiwiY29tcGFyZSI6IiIsInRlcm0iOiJQdWVydG8gUmljbywgVVNBIiwiZ2VvTG9jYXRpb25GYWNldCI6WyJQdWVydG8gUmljbywgVS5TLkEuIl0sImdlb0xvY2F0aW9uRmFjZXRSZWNlbnRzIjpbIlB1ZXJ0byBSaWNvLCBVLlMuQS4iXSwicGFnZSI6MSwibGltaXQiOjI1LCJvZmZzZXQiOjAsInNvcnQiOiIiLCJzb3J0VHlwZSI6IiIsImNsZWFyUGFyYW1zIjpmYWxzZSwia2V5d29yZCI6IiIsInBhZ2VUaXRsZSI6Imw6UHVlcnRvIFJpY28sIFUuUy5BLiIsInByYWN0aWNlQXJlYXMiOlsiQ29uc3VtZXIgTGF3Il19
accediendo a: https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto+Rico%2C+USA&params=eyJ0eXBlIjoicGVvcGxlIiwiY29tcGFyZSI6IiIsInRlcm0iOiJQdWVydG8gUmljbywgVVNBIiwiZ2VvTG9jYXRpb25GYWNldCI6WyJQdWVydG8gUmljbywgVS5TLkEuIl0sImdlb0xvY2F0aW9uRmFjZXRSZWNlbnRzIjpbIlB1ZXJ0byBSaWNvLCBVLlMuQS4iXSwicGFnZSI6MSwibGltaXQiOjI1LCJvZmZzZXQiOjAsInNvcnQiOiIiLCJzb3J0VHlwZSI6IiIsImNsZWFyUGFyYW1zIjpmYWxzZSwia2V5d29yZCI6IiIsInBhZ2VUaXRsZSI6Imw6UHVlcnRvIFJpY2

  2%|▏         | 2/91 [00:28<20:22, 13.74s/it]

...

paginas en esta url: 4
accediendo a: https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto+Rico%2C+USA&params=eyJ0eXBlIjoicGVvcGxlIiwiY29tcGFyZSI6IiIsInRlcm0iOiJQdWVydG8gUmljbywgVVNBIiwiZ2VvTG9jYXRpb25GYWNldCI6WyJQdWVydG8gUmljbywgVS5TLkEuIl0sImdlb0xvY2F0aW9uRmFjZXRSZWNlbnRzIjpbIlB1ZXJ0byBSaWNvLCBVLlMuQS4iXSwicGFnZSI6MSwibGltaXQiOjI1LCJvZmZzZXQiOjAsInNvcnQiOiIiLCJzb3J0VHlwZSI6IiIsImNsZWFyUGFyYW1zIjpmYWxzZSwia2V5d29yZCI6IiIsInBhZ2VUaXRsZSI6Imw6UHVlcnRvIFJpY28sIFUuUy5BLiIsInByYWN0aWNlQXJlYXMiOlsiSGVhbHRoIENhcmUiXX0%3D
https://www.martindale.com/organization/law-offices-of-stephenson-acquisto-colman-27628314/glendale-california-42115634-f/?pa=85
https://www.martindale.com/attorney/maria-l-quintana-1573303/
accediendo a: https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto+Rico%2C+USA&params=eyJ0eXBlIjoicGVvcGxlIiwiY29tcGFyZSI6IiIsInRlcm0iOiJQdWVydG8gUmljbywgVVNBIiwiZ2VvTG9jYXRpb25GYWNldCI6WyJQdWVydG8gUmljbywgVS5TLkEuIl0sImdlb0xvY2F0aW9uRmFjZXRSZ

 99%|█████████▉| 90/91 [37:36<00:20, 20.89s/it]

paginas en esta url: 17
accediendo a: https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto+Rico%2C+USA&params=eyJ0eXBlIjoicGVvcGxlIiwiY29tcGFyZSI6IiIsInRlcm0iOiJQdWVydG8gUmljbywgVVNBIiwiZ2VvTG9jYXRpb25GYWNldCI6WyJQdWVydG8gUmljbywgVS5TLkEuIl0sImdlb0xvY2F0aW9uRmFjZXRSZWNlbnRzIjpbIlB1ZXJ0byBSaWNvLCBVLlMuQS4iXSwicGFnZSI6MSwibGltaXQiOjI1LCJvZmZzZXQiOjAsInNvcnQiOiIiLCJzb3J0VHlwZSI6IiIsImNsZWFyUGFyYW1zIjpmYWxzZSwia2V5d29yZCI6IiIsInBhZ2VUaXRsZSI6Imw6UHVlcnRvIFJpY28sIFUuUy5BLiIsInByYWN0aWNlQXJlYXMiOlsiQ2l2aWwgTGl0aWdhdGlvbiJdfQ%3D%3D
accediendo a: https://www.martindale.com/search/attorneys-law-firms-articles/?term=Puerto+Rico%2C+USA&params=eyJ0eXBlIjoicGVvcGxlIiwiY29tcGFyZSI6IiIsInRlcm0iOiJQdWVydG8gUmljbywgVVNBIiwiZ2VvTG9jYXRpb25GYWNldCI6WyJQdWVydG8gUmljbywgVS5TLkEuIl0sImdlb0xvY2F0aW9uRmFjZXRSZWNlbnRzIjpbIlB1ZXJ0byBSaWNvLCBVLlMuQS4iXSwicGFnZSI6MSwibGltaXQiOjI1LCJvZmZzZXQiOjAsInNvcnQiOiIiLCJzb3J0VHlwZSI6IiIsImNsZWFyUGFyYW1zIjpmYWxzZSwia2V5d29yZCI6IiIsInBhZ2VUaXRsZSI6Imw6U

100%|██████████| 91/91 [38:12<00:00, 25.19s/it]

2473


In [281]:
with open("urls_region_PR.pkl", "wb") as f:
    pickle.dump(urls_per_filter, f)

In [ ]:
### Prueba scrapper 1 url
#url = "https://www.martindale.com/attorney/carlos-villegas-del-valle-301593670/#officeLocation"
#url = "https://www.martindale.com/attorney/rafael-g-rivera-rosario-1572639/"
url = "https://www.martindale.com/attorney/jairo-jay-mellado-villarreal-1572914/"

resultados = []
count = 1
driver = setup_driver()
wait = WebDriverWait(driver, 15)
info_attorney = {}
inicio = time.time()
for i in range(1):
    try:
        html = get_html(driver,url)
        soup = BeautifulSoup(html, 'html.parser')
        
        profile_content = soup.select_one('div.attorney-profile-content.profile-content')
        if not profile_content:
            print(f"Perfil no encontrado para {url}")
            pass

        # Obtencion de elementos princiaples de cada pagina
        attorney_card = profile_content.select_one(".masthead-content")
        firm = attorney_card.find("span").get_text()
        url_martindale_firm = attorney_card.find("a")["href"]

        # En caso de que no haya un link de firma, el primer link sera la caja de rewiew que redirige a la misma pagina, por eso el #
        if "#" in url_martindale_firm:
            url_martindale_firm = None

        # Estructura del diccionario
        info_attorney["url_martindale"] = url #URL interna de martindale para cada abogado
        info_attorney["name"] = profile_content.find('h1').get_text(strip=True) #Nombre del abogado
        info_attorney["firm"] = firm
        info_attorney["url_firm_martindale"] = url_martindale_firm

        # Busqueda de la sección de links de contacto, no todas las paginas tienen esta sección. por eso se usa try/except 
        try: 
            web_bar = profile_content.select_one("section.sticky-menu-all:nth-child(2) > nav:nth-child(1)")
            website_firm = web_bar.select('a')[1]["href"]
            info_attorney["website_firm"] = website_firm if website_firm else None
        except Exception as e:
            #print(f"Error al extraer website: {e}")
            info_attorney["website"] = ""

        #Extraccion de los tags relevantes de cada pagina
        main_content_table = soup.find_all("div", class_="small-12 columns")[0]
        for section_div in main_content_table.find_all("div", recursive=False):
            content_div = section_div.find("div", class_="toggle-area__content")
            h2_key = section_div.find("h2") #Titulo de sección por separado
            if h2_key and content_div:
                h2_key = h2_key.get_text(strip=True)
                if "Areas of Practice" in h2_key:
                    h2_key = "Areas of Practice"
                print(h2_key)
                info_attorney[h2_key] = "" #inicializar keys que se encuentran

                # Biography and practice areas 
                child_text = content_div.find('div', class_='truncate-text')
                if child_text:
                    info_attorney[h2_key] = child_text.get_text(strip=True)
                Aoe_area = content_div.find('div', class_='row collapse')
                if Aoe_area:
                    ul_info = Aoe_area.find('ul')
                    if ul_info:
                        items = [li.get_text(strip=True) for li in ul_info.find_all('li')]
                        info_attorney[h2_key] = ', '.join(items)

                # Education and credentials
                EnC_area = content_div.find_all(
                    lambda tag: tag.name == 'div' and 
                    all(cls in tag.get('class', []) for cls in ['row', 'collapse', 'experience-section', 'clearfix'])
                )
                for text in EnC_area:
                    text_info = text.get_text(strip=True)
                    for line in text_info.split('\n'):
                        if ':' in line:
                            k, v = map(str.strip, line.split(":", 1))
                            info_attorney[k] = v
                        else:
                            info_attorney[h2_key] = line.strip()
                
                # Peer rewiew - Pendiente
                PrA_section = content_div.find("section", class_="profile-peer-reviews")
                if PrA_section: 
                    rewiew_area = PrA_section.find('div', class_="review-top review-top-peer-reviews")
                    rewiew_list = PrA_section.find('div', class_="attorney-reviews-list attorney-peer-reviews-list")
                    if rewiew_area and rewiew_list:
                        text_rewiew_area = rewiew_area.find_all('span')[2].get_text().strip().splitlines()
                        stars = text_rewiew_area[0].split("/")[0].strip()   
                        reviews = text_rewiew_area[1].strip("()").split()[0]
                        info_attorney["profile_peer_review_star"] = stars
                        info_attorney["profile_peer_review_count"] = reviews
                    
                # Location
                firm_offices = content_div.find('ul', id='firmOffices')
                if firm_offices:
                    location_items = firm_offices.find_all('div', class_='office-address')
                    if location_items:
                        info_attorney["Location"] = location_items[0].get_text(strip=True)
        if info_attorney:
            resultados.append(info_attorney)
        count += 1 
        driver.quit()
        False
    except (NoSuchElementException, TimeoutException) as e:
        print(f"Error procesando {url}: {str(e)}")
        driver.quit()
    driver.quit()
    final = time.time() - inicio
    print(f"Tiempo por iteración: {final}")


# Corrida final, extracción de datos lista de urls final con filtros aplicados

In [ ]:
### carga de listado de urls
with open("urls_region_PR.pkl", "rb") as f:
    urls_total = pickle.load(f) # Carga de lista de diccionarios de abogados de primera iteración, ya la lista no es 0.
    # Esta variable va a ser mas general por lo que se va a emplear mas adelante. o asignar a la primera corrida. 

datos_PR = []
datos_parciales = []


In [101]:
len(urls_total)

2473

In [115]:
datos_parciales = scrape_abogados(urls_total[902:])
#Si falla, revisar en que iteración fue para continuar en la lista de urls_totales
# ultima 370


accediendo a 902: https://www.martindale.com/attorney/john-j-connors-1499939/
Failed to load page due to error: list index out of range
error en iter 902


In [116]:
datos_PR = datos_PR + datos_parciales

In [117]:
len(datos_PR)

900

In [ ]:
with open("data_attornies.pkl", "wb") as f:
    pickle.dump(datos_PR, f) # Almacenamiento de la lista de datos de la primera corrida (ya son los primeros 900) antes de seguir corriendo cargar en datos PR 
    # Para actualizar la iteración. 
